In [1]:
import pandas as pd
from pybatfish.client.commands import *
from pybatfish.datamodel import *
from pybatfish.question import bfq, list_questions, load_questions

pd.set_option("display.width", 300) 
pd.set_option("display.max_columns", 20) 
pd.set_option("display.max_rows", 1000) 
pd.set_option("display.max_colwidth", None)

# Configure all pybatfish loggers to use WARN level
import logging
logging.getLogger('pybatfish').setLevel(logging.WARN)

In [2]:
bf_session.host = 'localhost'

In [3]:
load_questions()


#### Routing and Forwarding Tables

* [Routes](#Routes)
* [BGP RIB](#BGP-RIB)
* [EVPN RIB](#EVPN-RIB)
* [Longest Prefix Match](#Longest-Prefix-Match)

In [4]:
bf_set_network('generate_questions')

'generate_questions'

In [5]:
bf_set_snapshot('generate_questions')

'generate_questions'

##### Routes

Returns routing tables.

Shows routes for specified RIB, VRF, and node(s).

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
nodes | Examine routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | 
network | Examine routes for networks matching this prefix. | str | True | 
protocols | Examine routes for protocols matching this specifier. | [RoutingProtocolSpec](../specifiers.md#routing-protocol-specifier) | True | 
vrfs | Examine routes on VRFs matching this name or regex. | str | True | 
rib | Only return routes from a given protocol RIB. | str | True | 

###### Invocation

In [6]:
result = bfq.routes().answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Node | Node | str
VRF | VRF name | str
Network | Network for this route | str
Next_Hop | Inferred hostname of the next hop | str
Next_Hop_IP | Route's Next Hop IP | str
Next_Hop_Interface | Route's Next Hop Interface | str
Protocol | Route's Protocol | str
Metric | Route's Metric | int
Admin_Distance | Route's Admin distance | int
Tag | Tag for this route | int

Print the first 5 rows of the returned Dataframe

In [7]:
result.head(5)

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,as1border1,default,1.0.1.1/32,None,AUTO/NONE(-1l),GigabitEthernet0/0,local,0,0,None
1,as2border1,default,2.23.11.0/24,as2core1,2.12.11.2,GigabitEthernet1/0,ospf,2,110,None
2,as2dist2,default,2.1.3.1/32,as2core1,2.23.12.2,GigabitEthernet1/0,ospf,3,110,None
3,as2core1,default,2.12.21.0/24,None,AUTO/NONE(-1l),GigabitEthernet1/0,connected,0,0,None
4,as2dist2,default,2.12.22.0/24,as2core2,2.23.22.2,GigabitEthernet0/0,ospf,2,110,None


Print the first row of the returned Dataframe

In [8]:
result.iloc[0]

Node                          as1border1
VRF                              default
Network                       1.0.1.1/32
Next_Hop                            None
Next_Hop_IP               AUTO/NONE(-1l)
Next_Hop_Interface    GigabitEthernet0/0
Protocol                           local
Metric                                 0
Admin_Distance                         0
Tag                                 None
Name: 0, dtype: object

In [9]:
bf_set_network('generate_questions')

'generate_questions'

In [10]:
bf_set_snapshot('generate_questions')

'generate_questions'

##### BGP RIB

Returns routes in the BGP RIB.

Shows BGP routes for specified VRF and node(s). This question is not available in Batfish containers on dockerhub prior to March 29, 2021.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
nodes | Examine routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | 
network | Examine routes for networks matching this prefix. | str | True | 
vrfs | Examine routes on VRFs matching this name or regex. | str | True | 

###### Invocation

In [11]:
result = bfq.bgpRib().answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Node | Node | str
VRF | VRF name | str
Network | Network for this route | str
Next_Hop_IP | Route's Next Hop IP | str
Next_Hop_Interface | Route's Next Hop Interface | str
Protocol | Route's Protocol | str
AS_Path | Route's AS path | str
Metric | Route's Metric | int
Local_Pref | Route's Local Preference | int
Communities | Route's List of communities | List of str
Origin_Protocol | Route's Origin protocol | str
Origin_Type | Route's Origin type | str
Originator_Id | Route's Originator ID | str
Cluster_List | Route's Cluster List | List of int
Tag | Tag for this route | int

Print the first 5 rows of the returned Dataframe

In [12]:
result.head(5)

,Node,VRF,Network,Next_Hop_IP,Next_Hop_Interface,Protocol,AS_Path,Metric,Local_Pref,Communities,Origin_Protocol,Origin_Type,Originator_Id,Cluster_List,Tag
0,as2border2,default,2.128.0.0/16,AUTO/NONE(-1l),null_interface,aggregate,,0,100,[],aggregate,igp,2.1.1.2,None,None
1,as2dist1,default,3.0.2.0/24,10.23.21.3,dynamic,ibgp,3,50,350,['3:2'],ibgp,igp,2.1.1.2,[33620482],None
2,as2dist2,default,3.0.1.0/24,10.23.21.3,dynamic,ibgp,3,50,350,['3:2'],ibgp,igp,2.1.1.2,[33620482],None
3,as2dist2,default,3.0.2.0/24,10.23.21.3,dynamic,ibgp,3,50,350,['3:2'],ibgp,igp,2.1.1.2,[33620482],None
4,as2dist1,default,3.0.2.0/24,10.23.21.3,dynamic,ibgp,3,50,350,['3:2'],ibgp,igp,2.1.1.2,[33620481],None


Print the first row of the returned Dataframe

In [13]:
result.iloc[0]

Node                      as2border2
VRF                          default
Network                 2.128.0.0/16
Next_Hop_IP           AUTO/NONE(-1l)
Next_Hop_Interface    null_interface
Protocol                   aggregate
AS_Path                             
Metric                             0
Local_Pref                       100
Communities                       []
Origin_Protocol            aggregate
Origin_Type                      igp
Originator_Id                2.1.1.2
Cluster_List                    None
Tag                             None
Name: 0, dtype: object

In [14]:
bf_set_network('generate_questions')

'generate_questions'

In [15]:
bf_set_snapshot('aristaevpn')

'aristaevpn'

##### EVPN RIB

Returns routes in the EVPN RIB.

Shows EVPN routes for specified VRF and node(s). This question is not available in Batfish containers on dockerhub prior to March 29, 2021.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
nodes | Examine routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | 
network | Examine routes for networks matching this prefix. | str | True | 
vrfs | Examine routes on VRFs matching this name or regex. | str | True | 

###### Invocation

In [16]:
result = bfq.evpnRib().answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Node | Node | str
VRF | VRF name | str
Network | Network for this route | str
Route_Distinguisher | Route distinguisher | str
Next_Hop_IP | Route's Next Hop IP | str
Next_Hop_Interface | Route's Next Hop Interface | str
Protocol | Route's Protocol | str
AS_Path | Route's AS path | str
Metric | Route's Metric | int
Local_Pref | Route's Local Preference | int
Communities | Route's List of communities | List of str
Origin_Protocol | Route's Origin protocol | str
Origin_Type | Route's Origin type | str
Originator_Id | Route's Originator ID | str
Cluster_List | Route's Cluster List | List of int
Tag | Tag for this route | int

Print the first 5 rows of the returned Dataframe

In [17]:
result.head(5)

,Node,VRF,Network,Route_Distinguisher,Next_Hop_IP,Next_Hop_Interface,Protocol,AS_Path,Metric,Local_Pref,Communities,Origin_Protocol,Origin_Type,Originator_Id,Cluster_List,Tag
0,dc1-svc3b,Tenant_A_WEB_Zone,192.168.254.4/32,192.168.255.5:15002,192.168.255.5,dynamic,bgp,65001 65102,0,100,['2:15002:15002'],bgp,egp,192.168.255.2,None,None
1,dc1-svc3a,Tenant_A_DB_Zone,192.168.254.4/32,192.168.255.5:15004,192.168.255.5,dynamic,bgp,65001 65102,0,100,['2:15004:15004'],bgp,egp,192.168.255.1,None,None
2,dc1-svc3b,Tenant_A_WEB_Zone,192.168.254.4/32,192.168.255.5:15002,192.168.255.5,dynamic,bgp,65001 65102,0,100,['2:15002:15002'],bgp,egp,192.168.255.1,None,None
3,dc1-leaf2b,Tenant_A_DB_Zone,192.168.254.6/32,192.168.255.7:15004,192.168.255.7,dynamic,bgp,65001 65103,0,100,['2:15004:15004'],bgp,egp,192.168.255.2,None,None
4,dc1-leaf2b,Tenant_A_DB_Zone,192.168.254.6/32,192.168.255.7:15004,192.168.255.7,dynamic,bgp,65001 65103,0,100,['2:15004:15004'],bgp,egp,192.168.255.1,None,None


Print the first row of the returned Dataframe

In [18]:
result.iloc[0]

Node                             dc1-svc3b
VRF                      Tenant_A_WEB_Zone
Network                   192.168.254.4/32
Route_Distinguisher    192.168.255.5:15002
Next_Hop_IP                  192.168.255.5
Next_Hop_Interface                 dynamic
Protocol                               bgp
AS_Path                        65001 65102
Metric                                   0
Local_Pref                             100
Communities              ['2:15002:15002']
Origin_Protocol                        bgp
Origin_Type                            egp
Originator_Id                192.168.255.2
Cluster_List                          None
Tag                                   None
Name: 0, dtype: object

In [19]:
bf_set_network('generate_questions')

'generate_questions'

In [20]:
bf_set_snapshot('generate_questions')

'generate_questions'

##### Longest Prefix Match

Returns routes that are longest prefix match for a given IP address.

Return longest prefix match routes for a given IP in the RIBs of specified nodes and VRFs.

###### Inputs

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
ip | IP address to run LPM on. | str | False | 
nodes | Examine routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | .*
vrfs | Examine routes on VRFs matching this name or regex. | str | True | .*

###### Invocation

In [21]:
result = bfq.lpmRoutes(ip='2.34.201.10').answer().frame()

###### Return Value

Name | Description | Type
--- | --- | ---
Node | Node where the route is present | str
VRF | VRF where the route is present | str
Ip | IP that was being matched on | str
Network | The longest-prefix network that matched | str
Num_Routes | Number of routes that matched (in case of ECMP) | int

Print the first 5 rows of the returned Dataframe

In [22]:
result.head(5)

,Node,VRF,Ip,Network,Num_Routes
0,as2border1,default,2.34.201.10,2.34.201.0/24,2
1,as2border2,default,2.34.201.10,2.34.201.0/24,2
2,as2core1,default,2.34.201.10,2.34.201.0/24,1
3,as2core2,default,2.34.201.10,2.34.201.0/24,1
4,as2dept1,default,2.34.201.10,2.34.201.0/24,1


Print the first row of the returned Dataframe

In [23]:
result.iloc[0]

Node             as2border1
VRF                 default
Ip              2.34.201.10
Network       2.34.201.0/24
Num_Routes                2
Name: 0, dtype: object